# Анализ базы данных

## Цель
Анализ информации о книгах, издательствах, авторах, а также пользовательские обзоры книг. Для формирования ценностного предложения для нового продукта

**Автор** Пинчук Ольга

## Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## 1. Подключение к БД

In [26]:
# Импорт библиотек
import pandas as pd
from sqlalchemy import create_engine 

In [27]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', 
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## 2. Сколько книг вышло после 1 января 2000 года

In [28]:
query = '''
SELECT 
COUNT(book_id)

FROM books
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine) 

,count
0,819


## 3. Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [29]:
query = '''
WITH books AS (
    SELECT book_id, 
    title 
    FROM books
),
g_ratings AS (
    SELECT 
    book_id,
    AVG(rating) AS avg_rating
    FROM ratings
    GROUP BY book_id
),
g_reviews AS (
    SELECT 
    book_id,
    COUNT(review_id) AS cnt_review
    FROM reviews
    GROUP BY book_id
)
SELECT 
books.title,
g_ratings.avg_rating,
g_reviews.cnt_review
FROM books
LEFT JOIN g_ratings ON books.book_id = g_ratings.book_id
LEFT JOIN g_reviews ON books.book_id = g_reviews.book_id
ORDER BY g_ratings.avg_rating DESC, g_reviews.cnt_review DESC

'''
pd.io.sql.read_sql(query, con = engine) 

,title,avg_rating,cnt_review
0,The Cat in the Hat and Other Dr. Seuss Favorites,5.00,NaN
1,A Dirty Job (Grim Reaper #1),5.00,4.0
2,Moneyball: The Art of Winning an Unfair Game,5.00,3.0
3,School's Out—Forever (Maximum Ride #2),5.00,3.0
4,Women,5.00,2.0
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,2.25,3.0
996,Drowning Ruth,2.00,3.0
997,His Excellency: George Washington,2.00,2.0
998,Junky,2.00,2.0


## 4. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [30]:
query = '''
WITH books AS (
    SELECT COUNT(book_id) AS cnt_books, 
    publisher_id 
    FROM books
    WHERE num_pages > 50
    GROUP BY publisher_id
)
SELECT
publishers.publisher,
books.cnt_books
FROM books
LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
ORDER BY books.cnt_books DESC
LIMIT 1 
'''
pd.io.sql.read_sql(query, con = engine) 

,publisher,cnt_books
0,Penguin Books,42


## 5. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [31]:
query = '''
WITH top_rat AS (
    SELECT
    book_id,
    COUNT(*) AS cnt_rating,
    AVG(rating) AS avg_rating
    FROM ratings
    GROUP BY book_id
    HAVING  COUNT(*) >= 50
)
SELECT
    authors.author,
    authors.author_id,
    AVG(top_rat.avg_rating) AS avg
FROM top_rat
LEFT JOIN books ON books.book_id=top_rat.book_id
LEFT JOIN authors ON authors.author_id = books.author_id
GROUP BY authors.author,authors.author_id
ORDER BY avg DESC
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine) 

,author,author_id,avg
0,J.K. Rowling/Mary GrandPré,236,4.283844


## 6. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [32]:
query = '''
WITH rat50 AS (
    SELECT
    username ,
    COUNT(rating) 
    FROM ratings
    GROUP BY username 
    HAVING  COUNT(rating) > 50
    ),
reviews AS (
    SELECT
    COUNT(review_id) as cntr,
    username
    FROM reviews
    GROUP BY username
)
SELECT 
AVG(reviews.cntr)
FROM reviews
RIGHT JOIN rat50 ON rat50.username=reviews.username 

'''
pd.io.sql.read_sql(query, con = engine) 

,avg
0,24.333333


## Общий вывод
- после 1 января 2000 года вышло 819 книг
- Для каждой книги расчитано кол-во обзорой и средняя оценка
- Издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books, и выпустило 42 книги
- Автор с самой высокой средней оценкой книг, среди книг с 50 и более оценками -J.K. Rowling/Mary GrandPré, средний рейтинг его книг - 4.28
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок -24
